# Import libraries

In [1]:
from math import sqrt
from numpy import array
from numpy import mean
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import explained_variance_score,max_error,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [2]:
def train_test_split(data, n_test):
    return data[:int(0.8*data.shape[0])], data[int(0.8*data.shape[0]):]

## LSTM 

In [3]:
def lstm_model(config):
    *_, n_features, n_input, n_nodes = config
    model = Sequential()
    model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dense(n_nodes, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
#     model.name="LSTM"
    return model

## Conv1D

In [4]:
def conv1d_model(config):
    *_, n_features, n_input, n_filters, n_kernel = config
    model = Sequential()
    model.add(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu', input_shape=(n_input, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
#     model.name="Conv1D"
    return model

In [5]:
def configuration(name):
    name = name.split("-")[0]
    
    if(name=="lstm"):
        n_input = [12]
        n_nodes = [100]
        n_epochs = [2, 3]
        n_batch = [1]
        n_diff = [12]
        n_features = [1]
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_nodes:
                                cfg = [i, j, k, l, m, n]
                                configs.append(cfg)
        return (configs, ["n_epochs","n_batch","n_diff","n_features","n_input","n_nodes"])
    
    if(name=="conv1d"):
        n_input = [12]
        n_filters = [64]
        n_kernel = [3]
        n_epochs = [50]
        n_batch = [1]
        n_diff = [12]
        n_features = [1]
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_filters:
                                for o in n_kernel:
                                    cfg = [i, j, k, l, m, n, o]
                                    configs.append(cfg)
        return (configs, ["n_epochs","n_batch","n_diff","n_features","n_input","n_filters","n_kernel" ])

In [6]:
series = read_csv('airline-passengers.csv', header=0, index_col=0)
data = series.values
n_test = 12

In [7]:
# transform list into supervised learning format
def series_to_supervised(data, n_in=1, n_out=1):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

In [18]:
def measure_error(actual, predicted):
    return [explained_variance_score(actual, predicted),max_error(actual, predicted),
            mean_absolute_error(actual, predicted),mean_squared_error(actual, predicted),
            sqrt(mean_squared_error(actual, predicted)),mean_squared_log_error(actual, predicted),
            median_absolute_error(actual, predicted),r2_score(actual, predicted)]
                                                    

In [9]:
def difference(data, order):
    return [data[i] - data[i - order] for i in range(order, len(data))]

In [10]:
def model_fit(train, config, create_model):
    # unpack config
    n_epochs, n_batch, n_diff,n_features,n_input,*_ = config
    # prepare data
    if n_diff > 0:
        train = difference(train, n_diff)
    # transform series into supervised format
    data = series_to_supervised(train, n_in=n_input)
    # separate inputs and outputs
    train_x, train_y = data[:, :-1], data[:, -1]
    # reshape input data into [samples, timesteps, features]
    train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
    # define model
#     model = lstm_model(config);
    model = create_model
    # fit
    model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
    return model

In [11]:
# forecast with the fit model
def model_predict(model, history, config, create_model):
    # unpack config
    _,_,n_diff,_,n_input,*_ = config
    # prepare data
    correction = 0.0
    if n_diff > 0:
        correction = history[-n_diff]
        history = difference(history, n_diff)
    x_input = array(history[-n_input:]).reshape((1, n_input, 1))
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return correction + yhat[0]

In [12]:
def walk_forward_validation(data, n_test, cfg, create_model,array_of_results):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model
    model = model_fit(train, cfg, create_model)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg, create_model)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_error(test, predictions)

    return error

In [28]:
def repeat_evaluate(data, config, n_test,create_model,array_of_results,keys, n_repeats=2):
    #MOŻE TU BYĆ PROBLEM W JAK ZMIENIMY n_repeats
    scores = [walk_forward_validation(data, n_test, config, create_model,array_of_results) for _ in range(n_repeats)]
    results ={}
    scores = zip(scores[0],scores[1])
    result = []
    for s in scores:
        print(s)
        result.append(mean(s))
        
    if "Model" not in results:
        results["Model"] = list()
    results["Model"].extend([create_model.name])

#     for k,c in zip(keys,config):
#         if str(k) not in results:
#             results[str(k)] = list()
#         results[str(k)].extend([c])

    if "EVC" not in results:
        results["EVC"] = list()
    results["EVC"].extend([result[0]])

    if "ME" not in results:
        results["ME"] = list()
    results["ME"].extend([result[1]])

    if "MAE" not in results:
        results["MAE"] = list()
    results["MAE"].extend([result[2]])

    if "MSE" not in results:
        results["MSE"] = list()
    results["MSE"].extend([result[3]])

    if "RMSE" not in results:
        results["RMSE"] = list()
    results["RMSE"].extend([result[4]])

    if "MSLE" not in results:
        results["MSLE"] = list()
    results["MSLE"].extend([result[5]])

    if "MED" not in results:
        results["MED"] = list()
    results["MED"].extend([result[6]])

    if "R2" not in results:
        results["R2"] = list()
    results["R2"].extend([result[7]])

  
    return results

In [29]:
cfg_list = []
cfg_list.append(lstm_model)
cfg_list.append(conv1d_model)

In [30]:
array_of_results = []
results = {}
array_of_models = []
for cfg_model in cfg_list:
    i=1
    results.clear()
    config = configuration(str(cfg_model.__name__.split("_")[0]))[0]
    for cfg in config:

        model = cfg_model(cfg)
        model.name=str(cfg_model.__name__.split("_")[0]+"-"+str(i))
        i +=1
        array_of_models.append(model)
        array_of_results.append(repeat_evaluate(data, cfg, n_test, model, array_of_results,configuration(str(cfg_model.__name__.split("_")[0]))[1]))

(0.9468484936904258, 0.9495282699686235)
(45.42891883850098, 46.50606346130371)
(14.539744574448157, 13.702353970757846)
(327.5957774082918, 322.3876166200161)
(18.099607106462056, 17.95515571138318)
(0.001789804833210754, 0.0017396186169801907)
(13.111427307128906, 10.212638854980469)
(0.9463352740112201, 0.9471884428884855)
(0.9540812535929154, 0.9491000836479098)
(49.197025299072266, 47.05603790283203)
(14.443612887941558, 15.0195731787846)
(328.82619177489346, 360.0596606310753)
(18.13356533544613, 18.97523809155172)
(0.0018049884265424155, 0.0018879721531011961)
(11.70190143585205, 11.492292404174805)
(0.94613371509505, 0.9410172402701712)
(0.9508769979189361, 0.9294352743531475)
(44.97792625427246, 47.946542739868164)
(14.193096160888672, 17.453691120805413)
(312.46199937713345, 435.66827310057687)
(17.67659467706191, 20.872668087730826)
(0.0016551772861188973, 0.0024422391103983278)
(13.12237548828125, 15.154914855957031)
(0.9488143964762357, 0.9286315022650361)


In [31]:
array_of_dataframes = []
for i in array_of_results:
    array_of_dataframes.append(pd.DataFrame.from_dict(i))

In [32]:
for i in array_of_dataframes:
    print(i)

    Model       EVC         ME        MAE         MSE       RMSE      MSLE  \
0  lstm-1  0.948188  45.967491  14.121049  324.991697  18.027381  0.001765   

         MED        R2  
0  11.662033  0.946762  
    Model       EVC         ME        MAE         MSE       RMSE      MSLE  \
0  lstm-2  0.951591  48.126532  14.731593  344.442926  18.554402  0.001846   

         MED        R2  
0  11.597097  0.943575  
      Model       EVC         ME        MAE         MSE       RMSE      MSLE  \
0  conv1d-1  0.940156  46.462234  15.823394  374.065136  19.274631  0.002049   

         MED        R2  
0  14.138645  0.938723  
